# 종합실습2_MNIST

## 1.환경준비

### (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.backend import clear_session
from keras.optimizers import Adam
from keras.datasets import mnist

from keras.callbacks import EarlyStopping

* 함수 만들기

In [ ]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['loss'], label='train_err', marker = '.')
    plt.plot(history['val_loss'], label='val_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

### (2) 데이터로딩

![](https://wikidocs.net/images/page/60324/mnist.png)

In [ ]:
# 케라스 데이터셋으로 부터 mnist 불러오기
(x_train, y_train), (x_val, y_val) = mnist.load_data()

In [ ]:
x_train.shape, y_train.shape

In [ ]:
class_names = ['0','1','2','3','4','5','6','7','8','9']

## 2 데이터 살펴보기

In [ ]:
# 아래 숫자를 바꿔가며 화면에 그려 봅시다.
n = 1

plt.figure()
plt.imshow(x_train[n], cmap=plt.cm.binary)
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])
plt.tight_layout()
plt.show()

## 3.데이터 준비

### (1) 데이터 2차원으로 펼치기

In [ ]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

In [ ]:
x_train = x_train.reshape(60000, -1)
x_val = x_val.reshape(10000, -1)

In [ ]:
x_train = x_train[:3000]
y_train = y_train[:3000]

x_val = x_val[:1000]
y_val = y_val[:1000]

In [ ]:
x_train.shape, x_val.shape

### (2) Scaling : Min-Max
* 0-255 값으로 되어 있는 데이터를 0-1사이 값으로 변환
* x_train, x_test를 그냥 255로 나누면 됨

In [ ]:
x_train = x_train / 255.
x_test = x_val / 255.

## 4.모델링 : 과적합

### (1) 학습

In [ ]:
nfeatures = x_train.shape[1]
nfeatures

In [ ]:
clear_session()

model = Sequential([Dense(10, input_shape = (nfeatures,), activation = 'softmax')])

model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy')
history = model.fit(x_train, y_train, epochs = 100, validation_split=0.2).history

### (2) 학습결과 그래프

In [ ]:
dl_history_plot(history)

* epoch가 약 40 근방에서 val_err가 다시 높아지기 시작합니다.

### (3) 예측 및 평가

In [ ]:
pred = model.predict(x_test)
pred_1 = pred.argmax(axis=1)

In [ ]:
print(confusion_matrix(y_val, pred_1))
print('='*70)
print(classification_report(y_val, pred_1))

## 5.모델링 : 과적합 방지

### (1) EarlyStopping 설정

In [ ]:
es = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 0)

### (2) 모델 선언 및 학습

In [ ]:
model = Sequential([Dense(10, input_shape = (nfeatures,), activation = 'softmax')])
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy')

history = model.fit(x_train, y_train, epochs = 100, validation_split = .2, 
                    callbacks = [es]).history

In [ ]:
dl_history_plot(history)

## 6.실습

* min_delta와 patience를 조절해 봅시다.

In [ ]:
# 아래 두 값 변경시켜보기
md = 0.01
pa = 5

es = EarlyStopping(monitor = 'val_loss', min_delta = md, patience = pa)

model = Sequential([Dense(10, input_shape = (nfeatures,), activation = 'softmax')])
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy')
history = model.fit(x_train, y_train, epochs = 100, validation_split = .2, callbacks = [es]).history

In [ ]:
dl_history_plot(history)

In [ ]:
pred = model.predict(x_test)
pred_1 = pred.argmax(axis=1)

In [ ]:
print(confusion_matrix(y_val, pred_1))
print('='*70)
print(classification_report(y_val, pred_1))

## 7.모델 저장하기

### (1) 모델 저장하기

* model.save(‘파일이름.h5’)
    * 딥러닝 모델의 메소드로 .save가 제공됩니다.
    * 파일이름.h5 파일이 저장됩니다. (h5 파일 포맷 : 하둡파일 형식)

In [ ]:
model.save('mnist_model.h5')

* 저장된 파일은 왼쪽 '파일'탭에서 확인 가능
* 파일을 다운로드 받을 수 있다.

### (2) 모델 로딩하기

* load_model 함수는 별도로 불러와야 합니다.
* 경로를 맞춰주고 h5 파일을 읽어오면 그대로 사용 가능합니다.

In [ ]:
from keras.models import load_model
model2 = load_model('mnist_model.h5')

* 불러온 모델 사용하기

In [ ]:
pred2 = model2.predict(x_test)

In [ ]:
pred2_1 = pred2.argmax(axis=1)

In [ ]:
print(accuracy_score(y_val,pred2_1))
print('-'*60)
print(confusion_matrix(y_val, pred2_1))
print('-'*60)
print(classification_report(y_val, pred2_1))